# Notebook de prise en main du package recordlinkage

Ce notebook vise à présenter le fonctionnement du package Splink et les possibilités qu'il offre. Le package Splink est un outil permettant de mettre en oeuvre des appariements probabilistes via la méthode de Fellegi-Sunter. Il se distingue par sa capacité à réaliser des appariements de fichiers très volumineux (jusqu'à plusieurs millions de lignes par fichier).

Les données utilisées sont des données synthétiques issues du [générateur de données synthétiques du package FEBRL](https://users.cecs.anu.edu.au/~Peter.Christen/Febrl/febrl-0.3/febrldoc-0.3/node70.html). Deux fichiers sont disponibles. Le premier fichier contient 5000 individus fictifs dont les traits d'identité (nom, prénom, date de naissance, etc.) proviennent de tables de fréquence tirés d'annuaires téléphoniques australiens. Le second fichier contient les mêmes 5000 individus, mais avec des erreurs sur les traits d'identité (substitution de caractères, suppression d'une valeur, remplacement par des variantes communes d'orthographe, etc.).

La documentation du package Splink est disponible [ici](https://moj-analytical-services.github.io/splink/).

Les différents termes techniques et notions évoquées dans ce notebook sont définis dans le [document de travail associé](https://www.insee.fr/fr/statistiques/fichier/version-html/7644535/M2023-03.pdf).

## Import des librairies et chargement des données

In [1]:
### Installation des packages splink et recordlinkage
!pip install splink
!pip install recordlinkage

In [2]:
### Import des librairies nécessaires
import pandas as pd
import recordlinkage
from recordlinkage.datasets import load_febrl4

from splink.duckdb.linker import DuckDBLinker
import splink.duckdb.comparison_library as cl
import splink.duckdb.comparison_template_library as ctl
from splink.duckdb.blocking_rule_library import block_on

In [3]:
### Chargement des données
dfA, dfB = load_febrl4()

In [4]:
n = len(dfA)

On va manipuler les données sous la forme de DataFrames de la librairie pandas.

On peut visualiser les 5 premières lignes des deux fichiers pour observer leur structure.

In [5]:
dfA.head()

,given_name,surname,street_number,address_1,address_2,suburb,postcode,state,date_of_birth,soc_sec_id
rec_id,,,,,,,,,,
rec-1070-org,michaela,neumann,8,stanley street,miami,winston hills,4223,nsw,19151111,5304218
rec-1016-org,courtney,painter,12,pinkerton circuit,bega flats,richlands,4560,vic,19161214,4066625
rec-4405-org,charles,green,38,salkauskas crescent,kela,dapto,4566,nsw,19480930,4365168
rec-1288-org,vanessa,parr,905,macquoid place,broadbridge manor,south grafton,2135,sa,19951119,9239102
rec-3585-org,mikayla,malloney,37,randwick road,avalind,hoppers crossing,4552,vic,19860208,7207688


In [6]:
dfB.head()

,given_name,surname,street_number,address_1,address_2,suburb,postcode,state,date_of_birth,soc_sec_id
rec_id,,,,,,,,,,
rec-561-dup-0,elton,NaN,3,light setreet,pinehill,windermere,3212,vic,19651013,1551941
rec-2642-dup-0,mitchell,maxon,47,edkins street,lochaoair,north ryde,3355,nsw,19390212,8859999
rec-608-dup-0,NaN,white,72,lambrigg street,kelgoola,broadbeach waters,3159,vic,19620216,9731855
rec-3239-dup-0,elk i,menzies,1,lyster place,NaN,northwood,2585,vic,19980624,4970481
rec-2886-dup-0,NaN,garanggar,NaN,may maxwell crescent,springettst arcade,forest hill,2342,vic,19921016,1366884


## Appariement étape par étape

Sous le capot, Splink fonctionne grâce à des opérations SQL. Il est possible d'utiliser différents moteurs pour effectuer ces opérations. Ce tutoriel présente d'abord le plus léger et facile à utiliser, DuckDB. Il est conseillé pour des fichiers pouvant aller jusqu'à 1 à 2 millions de lignes. Pour des fichiers plus volumineux, il faudra utiliser un moteur de traitement adapté aux très grands volumes de données, comme Spark. Un exemple d'appariement avec Splink reposant sur Spark est proposé dans ce notebook, après l'appariement avec DuckDB. Le passage de l'un à l'autre est très aisé puisque la syntaxe et les noms de fonctions restent les même.

### Préparation des données

En général, la première étape d'un appariement consiste en une succession de traitements pour nettoyer les données. Ici, les données synthétiques utilisées sont déjà dans un format satisfaisant (uniquement des caractères minuscules, pas d'accents, etc.). On effectue simplement une opération pour que l'identifiant des individus soit une colonne et non un index pandas.

In [7]:
dfA = dfA.reset_index()
dfB = dfB.reset_index()

### Définition des principaux paramètres

On définit ici les principaux paramètres de l'appariement sous la forme d'un gros dictionnaire. Les règles de blocage et les méthodes de comparaison sont notamment définies ici. Aucun calcul n'intervient à ce stade en revanche, le dictionnaire sera utilisé comme argument d'une fonction par la suite.

#### Réduction du nombre de paires : blocage

L'ensemble des paires possibles est le produit cartésien des deux fichiers : chaque individu du fichier A peut être comparé à chaque individu du fichier B. En pratique, il est souvent impossible de conserver toutes les paires, car le produit cartésien devient rapidement colossal lorsque la taille des fichiers augmente. Il faut donc trouver un moyen de filtrer. L'approche classique, et celle proposée par Splink, est le blocage.

Le blocage consiste à conserver uniquement les paires qui correspondent sur une variable ou une sélection de variables, appelées "clés de blocage". Toutes les autres paires sont d'office rejetées, elle ne seront pas comparées en détail. Une bonne clé de blocage doit être assez discriminante pour réduire largement la dimension, et de très bonne qualité pour éviter de supprimer des paires d'individus identiques.

In [8]:
### Différents exemples de règles de blocage

# Blocage simple sur le code postal
blocking_rules_postcode = [
        "l.postcode = r.postcode",
    ]

# Blocage sur le code postal OU l'année de naissance
blocking_rules_postcode_or_yearofbirth = [
        "l.postcode = r.postcode",
        "LEFT(l.date_of_birth, 4) = LEFT(r.date_of_birth, 4)",
    ]

# Blocage sur le code postal ET l'année de naissance
blocking_rules_postcode_and_yearofbirth = [
        "l.postcode = r.postcode and LEFT(l.date_of_birth, 4) = LEFT(r.date_of_birth, 4)",
    ]

La méthode *count_num_comparisons_from_blocking_rule* permet de calculer le nombre de paires induites par une règle de blocage, ce qui est pratique pour évaluer sa pertinence et si elle doit être remplacée ou complétée.

In [9]:
# Initialisation de l'objet Linker, plus de détails sur ce point plus loin dans le notebook
linker = DuckDBLinker([dfA, dfB], {"link_type": "link_only", "unique_id_column_name": "rec_id"})

print("Nombre de paires conservées - blocage simple sur le code postal : "
f"{linker.count_num_comparisons_from_blocking_rule(' or '.join(blocking_rules_postcode))}")

print("Nombre de paires conservées - blocage sur le code postal OU l'année de naissance : " 
f"{linker.count_num_comparisons_from_blocking_rule(' or '.join(blocking_rules_postcode_or_yearofbirth))}")

print("Nombre de paires conservées - blocage sur le code postal ET l'année de naissance : " 
f"{linker.count_num_comparisons_from_blocking_rule(' or '.join(blocking_rules_postcode_and_yearofbirth))}")

Nombre de paires conservées - blocage simple sur le code postal : 28609


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

Nombre de paires conservées - blocage sur le code postal OU l'année de naissance : 265905
Nombre de paires conservées - blocage sur le code postal ET l'année de naissance : 4052


Le package propose également une méthode pour visualiser le nombre de paires apportées par chaque nouvelle règle de blocage, dans le cas où plusieurs sont utilisées.

In [10]:
linker.cumulative_num_comparisons_from_blocking_rules_chart(blocking_rules_postcode_or_yearofbirth)

alt.Chart(...)

Pour la suite de ce tutoriel, nous allons conserver les paires issues du blocage simple sur le code postal

#### Comparaison des champs

L'étape suivante consiste à choisir la façon dont vont être comparées les paires conservées champ par champ. Le package propose différents moyens de définir ces comparaisons, qui sont détaillés au sein de [la section dédiée dans documentation](https://moj-analytical-services.github.io/splink/topic_guides/comparisons/customising_comparisons.html). Il existe des méthodes de comparaison prédéfinies adaptées à différents types de champs (date, nom, code postal, etc.), mais il est souvent intéressant de les paramétrer soi-même pour tenir compte des spécifités des fichiers à apparier.

Dans tous les cas, dans un appariement probabiliste, il faut utiliser des mesures de comparaison discrètes. Cela n'empêche pas de se servir des mesures de similarité classiques telles que Jaro-Winkler, mais il faut transformer le résultat pour obtenir un nombre fixe de modalités.

Dans cet exemple, les champs `date_of_birth`, `suburb` et `state` sont comparés de façon exacte, avec un résultat binaire. Les champs `given_name` et `surname` sont comparés via la similarité de Jaro-Winkler, en utilisant différents seuils (plus de détails ci-dessous). Enfin, c'est la distance de Levenshtein qui est utilisée pour le champ `address_1`.

On n'effectue aucune comparaison sur la variable `postcode` ici, puisqu'elle est la seule clé de blocage : ainsi, toutes les paires retenues présentent forcément le même code postal, cette variable ne porte plus d'information.

Par ailleurs, le paramètre `term_frequency_adjustments` est utilisé ici pour prendre en compte la fréquence des modalités pour chaque champ. Par exemple, une correspondance sur un nom rare fera augmenter la probabilité de façon plus significative qu'une correspondance sur un nom plus répandu. (Plus de détails sur cet ajustement dans [la documentation](https://moj-analytical-services.github.io/splink/topic_guides/comparisons/term-frequency.html))

In [11]:
### Définition de la méthode de comparaison pour chaque champ
comparisons_list = [
        cl.jaro_winkler_at_thresholds("given_name", [0.95, 0.88], term_frequency_adjustments = True),
        cl.jaro_winkler_at_thresholds("surname", [0.95, 0.88], term_frequency_adjustments = True),
        cl.exact_match("date_of_birth", term_frequency_adjustments=True),
        cl.exact_match("suburb", term_frequency_adjustments=True),
        cl.exact_match("state", term_frequency_adjustments=True),
        cl.levenshtein_at_thresholds("address_1", [3, 6], term_frequency_adjustments=True),
    ]

Splink possède une fonction pour transformer le code utilisé pour définir une méthode de comparaison en une description plus facilement compréhensible, ce qui est pratique pour vérifier que la méthode de comparaison définie est conforme à l'intention initiale.

En utilisant cette fonction sur la méthode de comparaison définie pour le prénom par exemple, on comprend qu'elle induit 5 niveaux : 
- lorsque l'une des deux valeurs est manquante,
- les correspondances exactes,
- les valeurs différentes mais dont la similarité de Jaro-Winkler est supérieure à 0,95,
- les valeurs différentes mais dont la similarité de Jaro-Winkler est comprise entre à 0,95 et 0,88,
- les valeurs dont la similarité de Jaro-Winkler est inférieure à 0,88.

In [12]:
first_name_comparison = cl.jaro_winkler_at_thresholds("given_name", [0.95, 0.88])
print(first_name_comparison.human_readable_description)

Comparison 'Exact match vs. Given_Name within jaro_winkler_similarity thresholds 0.95, 0.88 vs. anything else' of "given_name".
Similarity is assessed using the following ComparisonLevels:
    - 'Null' with SQL rule: "given_name_l" IS NULL OR "given_name_r" IS NULL
    - 'Exact match' with SQL rule: "given_name_l" = "given_name_r"
    - 'Jaro_winkler_similarity >= 0.95' with SQL rule: jaro_winkler_similarity("given_name_l", "given_name_r") >= 0.95
    - 'Jaro_winkler_similarity >= 0.88' with SQL rule: jaro_winkler_similarity("given_name_l", "given_name_r") >= 0.88
    - 'All other comparisons' with SQL rule: ELSE



Voici un autre exemple avec la méthode de comparaison définie pour l'adresse, en mettant des seuils à 3 et à 6 pour la distance de Levenshtein.

In [13]:
address_comparison = cl.levenshtein_at_thresholds("address_1", [3, 6])
print(address_comparison.human_readable_description)

Comparison 'Exact match vs. Address_1 within levenshtein thresholds 3, 6 vs. anything else' of "address_1".
Similarity is assessed using the following ComparisonLevels:
    - 'Null' with SQL rule: "address_1_l" IS NULL OR "address_1_r" IS NULL
    - 'Exact match' with SQL rule: "address_1_l" = "address_1_r"
    - 'Levenshtein <= 3' with SQL rule: levenshtein("address_1_l", "address_1_r") <= 3
    - 'Levenshtein <= 6' with SQL rule: levenshtein("address_1_l", "address_1_r") <= 6
    - 'All other comparisons' with SQL rule: ELSE



#### Définition du dictionnaire de paramètres

Il est maintenant temps de définir le dictionnaire de paramètres, en incorporant les règles de blocage et les méthodes de comparaison définies précédemment. La liste complète des paramètres modifiables dans ce dictionnaires est disponible [sur cette page](https://moj-analytical-services.github.io/splink/settings_dict_guide.html).

In [14]:
linkage_settings = {
    # Le paramètre "link_type" sert à préciser le type de tâche à effectuer
    # Ici, il s'agit d'un appariement entre deux fichiers ("link_only") et 
    # non d'un dédoublonnage ("deduplication")
    "link_type": "link_only",
    
    # Liste des règles de blocage
    "blocking_rules_to_generate_predictions": blocking_rules_postcode,
    
    # Liste spécifiant la façon dont les paires doivent être comparées
    "comparisons": comparisons_list,

    # Nom de la colonne contenant l'identifiant unique (par défaut il s'agit de "unique_id")
    "unique_id_column_name": "rec_id"
}

### Estimation des paramètres

In [15]:
linker = DuckDBLinker([dfA, dfB], linkage_settings)

[Traduction à venir]

It is possible to estimate u probabilities beforehand using random sampling instead of estimating them along with the m probabilities with the EM algorithm. The main advantage is that it is less computationnaly intensive. Extract of the documentation explaining how this procedure works :

*The u parameters represent the proportion of record comparisons that fall into each comparison level amongst truly non-matching records.*

*This procedure takes a sample of the data and generates the cartesian product of pairwise record comparisons amongst the sampled records. The validity of the u values rests on the assumption that the resultant pairwise comparisons are non-matches (or at least, they are very unlikely to be matches). For large datasets, this is typically true.*

In [16]:
linker.estimate_u_using_random_sampling(max_pairs=1e6)

----- Estimating u probabilities using random sampling -----

Estimated u probabilities using random sampling

Your model is not yet fully trained. Missing estimates for:
    - given_name (no m values are trained).
    - surname (no m values are trained).
    - date_of_birth (no m values are trained).
    - suburb (no m values are trained).
    - state (no m values are trained).
    - address_1 (no m values are trained).


[Traduction à venir]

**Expectation-Maximisation algorithm**

Estimation of the m probabilities is typically done with the Expectation-Maximisation algorithm. Is is an unsupervised approach, meaning it does not require labeled pairs.

If the estimation of u probabilities has not been done beforehand, it is possible to estimate them here by setting the parameter `fix_u_probabilities` to False.

**Specific blocking rules for training**

The function takes as argument a list of blocking rules. Estimation is done using only the pairs remaining after applying these blockings rules.

Extract of the documentation explaining this parameter and why these blocking tules might be different from the blocking rules to generate predictions:

*The purpose of this blocking rule is to reduce the number of pairwise generated to a computationally-tractable number to enable the expectation maximisation algorithm to work.*

*The expectation maximisation algorithm seems to work best when the pairwise record comparisons are a mix of anywhere between around 0.1% and 99.9% true matches. It works less effectively if there are very few examples of either matches or non-matches. It works less efficiently if there is a huge imbalance between the two (e.g. a billion non matches and only a hundred matches).*

*It does not matter if this blocking rule excludes some true matches - it just needs to generate examples of matches and non matches.*

*Since they serve different purposes, the blocking rules most appropriate to use with blocking_rules_to_generate_predictions will often be different to those for estimate_parameters_using_expectation_maximisation, but it is also common for the same rule to be used in both places.*

**Estimating the m probabilities in several steps**

If for instance an exact match on `postcode` is used as training blocking rule, it is impossible to estimate the m probability for this field because all pairs used to estimate the model parameters match on postcode. By extension, Splink disables estimation of the m probability for all the fields used in the training blocking rules. For example, with the rule `l.postcode = r.postcode or LEFT(l.date_of_birth, 4) = LEFT(r.date_of_birth, 4)`, Splink will not provide an estimate for the fields `postcode` and `date_of_birth`.

Splink's documentation suggests to estimate the m probabilities sequentially with different training blocking rules, so that an estimate can be obtained for every field. Below is an example of an estimation procedure using this strategy, with two steps.

In [17]:
session_dob = linker.estimate_parameters_using_expectation_maximisation(block_on("date_of_birth"))
session_first_name = linker.estimate_parameters_using_expectation_maximisation(block_on("given_name"))


----- Starting EM training session -----

Estimating the m probabilities of the model by blocking on:
l."date_of_birth" = r."date_of_birth"

Parameter estimates will be made for the following comparison(s):
    - given_name
    - surname
    - suburb
    - state
    - address_1

Parameter estimates cannot be made for the following comparison(s) since they are used in the blocking rules: 
    - date_of_birth

Iteration 1: Largest change in params was 0.538 in probability_two_random_records_match
Iteration 2: Largest change in params was 0.0261 in probability_two_random_records_match
Iteration 3: Largest change in params was 0.000968 in probability_two_random_records_match
Iteration 4: Largest change in params was 4.21e-05 in probability_two_random_records_match

EM converged after 4 iterations

Your model is not yet fully trained. Missing estimates for:
    - date_of_birth (no m values are trained).

----- Starting EM training session -----

Estimating the m probabilities of the model 

### Classification des paires

[Traduction à venir]

Once paramaters (m and u probabilities) have been estimated, the `predict` method enables to compute a match probability for every pair remaining after applying the `blocking_rules_to_generate_predictions` (which were defined in the settings dictionary).

In [22]:
# Optionally, a threshold_match_probability can be provided, which will drop any row 
# where the predicted score is below the threshold
results = linker.predict(threshold_match_probability=0.5)

In [23]:
results_pandas = results.as_pandas_dataframe()

In [24]:
results_pandas.shape

(4146, 26)

### Résolution des conflits

[Traduction à venir]

If both files are assumed to be free of duplicates, there is a constraint of one-to-one matching.

An additional step needs to be performed on the linkage results in order to remove potential conflicts that would violate this constraint.

Below is an SQL query performing this operation. It always keeps the pair with the highest probability.

In [25]:
sql = f"""
with ranked as

(
select *,
row_number() OVER (
    PARTITION BY rec_id_l order by match_weight desc
    ) as row_number
from {results.physical_name}
)

select *
from ranked
where row_number = 1


"""
results = linker.query_sql(sql)

In [26]:
results

,match_weight,match_probability,source_dataset_l,source_dataset_r,rec_id_l,rec_id_r,given_name_l,given_name_r,gamma_given_name,surname_l,...,gamma_suburb,state_l,state_r,gamma_state,address_1_l,address_1_r,gamma_address_1,postcode_l,postcode_r,row_number
0,35.182011,1.000000,__splink__input_table_0,__splink__input_table_1,rec-105-org,rec-105-dup-0,breony,breony,3,webb,...,1,nsw,nsw,1,lewis luxton avenue,lewis luxton avenue,3,4216,4216,1
1,10.426069,0.999274,__splink__input_table_0,__splink__input_table_1,rec-1062-org,rec-1062-dup-0,dale,dale,3,dietrich,...,-1,nsw,nws,0,sturt avenue,sturt avenue,3,6110,6110,1
2,28.974042,1.000000,__splink__input_table_0,__splink__input_table_1,rec-1099-org,rec-1099-dup-0,kate,kawe,0,garden,...,1,nsw,nsw,1,bettington circuit,bettington circuit,3,3121,3121,1
3,11.702925,0.999700,__splink__input_table_0,__splink__input_table_1,rec-1123-org,rec-1123-dup-0,gabrielle,gabrielle,3,rees,...,0,vic,vic,1,julia flynn avenue,julia flynn avenue,3,4300,4300,1
4,35.057483,1.000000,__splink__input_table_0,__splink__input_table_1,rec-1127-org,rec-1127-dup-0,william,william,3,bobbitt,...,1,nsw,nsw,1,woralul street,woralu lstreet,2,2340,2340,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4141,23.423262,1.000000,__splink__input_table_0,__splink__input_table_1,rec-94-org,rec-94-dup-0,james,james,3,compton,...,1,nsw,nsw,1,brewster place,landsborough street,0,2234,2234,1
4142,17.455669,0.999994,__splink__input_table_0,__splink__input_table_1,rec-959-org,rec-959-dup-0,zac,zac,3,hand,...,0,sa,sa,1,kambalda crescent,kambalda crescent,3,4500,4500,1
4143,35.894870,1.000000,__splink__input_table_0,__splink__input_table_1,rec-976-org,rec-976-dup-0,isabelle,isabelle,3,campbell,...,1,qld,qld,1,jukes street,jukes street,3,4811,4811,1
4144,31.630634,1.000000,__splink__input_table_0,__splink__input_table_1,rec-988-org,rec-988-dup-0,jack,jack,3,andrae,...,1,vic,vic,1,cleland street,clelandojstreet,2,6025,6025,1


### Évaluation de la qualité

In [27]:
def compute_performance_metrics_FEBRL(results, dataset_size):
    """
    Compute performance metrics of a record linkage process on FEBRL synthetic data.
    The assumption is that the size of the two datasets is the same and every record 
    from dataset A has exactly one match in dataset B.

            Parameters:
                    results (pandas DataFrame): Output from the linkage process
                    dataset_size (int): Length of both datasets to be linked

            Returns:
                    performance_metrics (tuple): Tuple of metrics (TP, TN, FP, FN, precision, recall, F-measure)
    """
    results['actual'] = (results['rec_id_l'].str.extract(r'(rec-\d+)') 
                                == results['rec_id_r'].str.extract(r'(rec-\d+)'))
    TP = sum(results['actual'])
    FP = sum(~results['actual'])
    #Pairs that were removed in the indexing phase must be taken into account to compute True and False negatives
    FN = dataset_size - TP
    TN = dataset_size*dataset_size - TP - FN - FP

    precision = TP / (TP + FP)
    recall = TP / (TP + FN)
    Fscore = 2 * precision * recall / (precision + recall)
    performance_metrics = (TP, TN, FP, FN, precision, recall, Fscore)
    return(performance_metrics)

def print_performance_metrics(linkage_output, dataset_size):
    """
    Prints performance metrics of a record linkage process on synthetic data.
    The assumption is that the size of the two datasets is the same and every record 
    from dataset A has exactly one match in dataset B.

            Parameters:
                    results (pandas DataFrame): Output from the linkage process
                    dataset_size (int): Length of both datasets to be linked

            Returns:
                    None
    """
    TP, TN, FP, FN, precision, recall, Fscore = compute_performance_metrics_FEBRL(results, dataset_size)
    print(f"Vrais positifs : {TP:,}".replace(',', ' '))
    print(f"Vrais négatifs : {TN:,}".replace(',', ' '))
    print(f"Faux positifs : {FP:,}".replace(',', ' '))
    print(f"Faux négatifs : {FN:,}".replace(',', ' '))
    print(f"Précision : {precision:.4}")
    print(f"Rappel : {recall:.4}")
    print(f"F-mesure : {Fscore:.4}")

In [28]:
print_performance_metrics(results, n)

Vrais positifs : 4 146
Vrais négatifs : 24 995 000
Faux positifs : 0
Faux négatifs : 854
Précision : 1.0
Rappel : 0.8292
F-mesure : 0.9066


On peut effectuer deux constats sur les résultats. 
- D'abord, il n'y a aucun faux positif ou presque, probablement dû au fait que les erreurs générées sur le fichier B sont assez légères et ne perturbent donc pas trop l'algorithme d'appariement.
- Du côté des faux négatifs, c'est moins satisfaisant : on rate beaucoup de paires d'individus identiques. La raison de ce défaut est sans doute le blocage un peu trop strict qui a été choisi, puisqu'on n'utilise que le code postal comme clé de blocage. Ainsi, tous les individus qui présentent une erreur sur le code postal dans le fichier B n'ont aucune chance d'être bien appariés, quelle que soit la qualité des données sur les autres champs identifiants. On pourrait donc améliorer l'appariement en utilisant plusieurs clés de bocage, afin de ne pas mettre trop de poids sur une seule variable.

Quoi qu'il en soit, ces résultats sont propres aux deux fichiers appariés. Un autre jeu de fichiers, avec des données de qualité différente et des types d'erreurs différents, donnerait un tout autre résultat.

In [29]:
linker.missingness_chart()

alt.LayerChart(...)

## Exemple de code complet pour un appariement

In [1]:
### Import des librairies nécessaires
import pandas as pd
import recordlinkage
from recordlinkage.datasets import load_febrl4

from splink.duckdb.linker import DuckDBLinker
import splink.duckdb.comparison_library as cl
import splink.duckdb.comparison_template_library as ctl
from splink.duckdb.blocking_rule_library import block_on

### Chargement et préparation des données
dfA, dfB = load_febrl4()
dfA = dfA.reset_index()
dfB = dfB.reset_index()

### Définition des principaux paramètres

# Blocage simple sur le code postal
blocking_rules_postcode = [
        "l.postcode = r.postcode",
    ]

# Définition de la méthode de comparaison pour chaque champ
comparisons_list = [
        cl.jaro_winkler_at_thresholds("given_name", [0.95, 0.88], term_frequency_adjustments = True),
        cl.jaro_winkler_at_thresholds("surname", [0.95, 0.88], term_frequency_adjustments = True),
        cl.exact_match("date_of_birth", term_frequency_adjustments=True),
        cl.exact_match("suburb", term_frequency_adjustments=True),
        cl.exact_match("state", term_frequency_adjustments=True),
        cl.levenshtein_at_thresholds("address_1", [3, 6], term_frequency_adjustments=True),
    ]

# Définition du dictionnaire de paramètres
linkage_settings = {
    # Le paramètre "link_type" sert à préciser le type de tâche à effectuer
    # Ici, il s'agit d'un appariement entre deux fichiers ("link_only") et 
    # non d'un dédoublonnage ("deduplication")
    "link_type": "link_only",
    
    # Liste des règles de blocage
    "blocking_rules_to_generate_predictions": blocking_rules_postcode,
    
    # Liste spécifiant la façon dont les paires doivent être comparées
    "comparisons": comparisons_list,

    # Nom de la colonne contenant l'identifiant unique (par défaut il s'agit de "unique_id")
    "unique_id_column_name": "rec_id"
}

### Estimation des paramètres

# Initialisation de l'objet Linker
linker = DuckDBLinker([dfA, dfB], linkage_settings)

# Estimation des probabilités u (par échantillonnage aléatoire)
linker.estimate_u_using_random_sampling(max_pairs=1e6)

# Estimation des probabilités m (par l'algorithme EM)
session_dob = linker.estimate_parameters_using_expectation_maximisation(block_on("date_of_birth"))
session_first_name = linker.estimate_parameters_using_expectation_maximisation(block_on("given_name"))

### Classification des paires
results = linker.predict(threshold_match_probability=0.5)

### Résolution des conflits
sql = f"""
with ranked as

(
select *,
row_number() OVER (
    PARTITION BY rec_id_l order by match_weight desc
    ) as row_number
from {results.physical_name}
)

select *
from ranked
where row_number = 1


"""
results = linker.query_sql(sql)

----- Estimating u probabilities using random sampling -----


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))


Estimated u probabilities using random sampling

Your model is not yet fully trained. Missing estimates for:
    - given_name (no m values are trained).
    - surname (no m values are trained).
    - date_of_birth (no m values are trained).
    - suburb (no m values are trained).
    - state (no m values are trained).
    - address_1 (no m values are trained).

----- Starting EM training session -----

Estimating the m probabilities of the model by blocking on:
l."date_of_birth" = r."date_of_birth"

Parameter estimates will be made for the following comparison(s):
    - given_name
    - surname
    - suburb
    - state
    - address_1

Parameter estimates cannot be made for the following comparison(s) since they are used in the blocking rules: 
    - date_of_birth

Iteration 1: Largest change in params was 0.544 in probability_two_random_records_match
Iteration 2: Largest change in params was 0.0264 in probability_two_random_records_match
Iteration 3: Largest change in params was 0.00

## Appariement avec Spark

À venir

## Exercice

À venir